##### Imports

In [77]:
import grobid_tei_xml
import pandas as pd
import xml.etree.ElementTree as ET
import json
import re

In [78]:
file = "g.grobid.tei.xml"
with open(file, 'r') as xml_file:
    grob_doc = grobid_tei_xml.parse_document_xml(xml_file.read().encode())
    et_doc = ET.parse(file)

#Get AnyStyle Dictionary
with open('ref.json', 'r') as json_file:
    AnyStyle_data = json.load(json_file)

#Extract Titles
body = grob_doc
root = et_doc.getroot()

# Get Grobid Dictionary
grobid_data = [p.to_dict() for p in grob_doc.citations]

##### Find in text references

In [79]:
df = pd.DataFrame(columns = ['reference', 'paragraph', "context", "heading"])

curr_heading = ""
curr_paragraph = ""
context = "None"
curr_ref = ""
count = 0

for element in root.iter():
    if(element.tag.endswith("head")):
        if(element.text != None):
            curr_heading = element.text

    if(element.tag.endswith("p") or element.tag.endswith("figDesc")):
        curr_paragraph = element.text

    if(element.tag.endswith("ref")):
        curr_ref = element.text
        df.loc[count] = [curr_ref, curr_paragraph, context, curr_heading]
        count= count+1

df = df.dropna()


Clean DataFrame

##### Connect in text references with Bibliography

In [92]:
def check_format(string):
    pattern = r'^\[\d+\]$'
    match = re.match(pattern, string)
    return bool(match)

def find_string_in_nested_dicts(list_of_dicts, key_to_search, string_to_find):
    # Iterate through each dictionary in the list
    for dictionary in list_of_dicts:
        # Check if the key exists and its value is a list
        if key_to_search in dictionary and isinstance(dictionary[key_to_search], list):
            # Iterate through each nested dictionary in the list
            for nested_dict in dictionary[key_to_search]:
                # Check if the string exists in any of the nested dictionaries
                if string_to_find in nested_dict.values():
                    return True  # Return True if found
    return False  # Return False if not found

def compare_bib(ref):
    clean_ref = re.sub(r'[^\w\s]', '', ref)
    try:
        if check_format(ref):
            return True
        elif(re.match(r'^\d{4}$', clean_ref)):
            return True
        else:
            # Remove punctuation using regular expressions
            text = clean_ref.split()[0]
            return find_string_in_nested_dicts(grobid_data, 'authors', text)
    except:
        return False

df= df[df['reference'].apply(compare_bib)]
df.head(10)



,reference,paragraph,context,heading
9,"(Gay & Keith, 1991;",Despite the establishment of Toll as a regulat...,None,A brief history of Toll
10,"Heguy et al., 1992)",Despite the establishment of Toll as a regulat...,None,A brief history of Toll
11,"(Letsou et al., 1991;",Despite the establishment of Toll as a regulat...,None,A brief history of Toll
12,"Shelton et al., 1993;",Despite the establishment of Toll as a regulat...,None,A brief history of Toll
13,"Grosshans et ai., 1994)",Despite the establishment of Toll as a regulat...,None,A brief history of Toll
14,"(Geisler et al., 1992)",Despite the establishment of Toll as a regulat...,None,A brief history of Toll
15,"(Ip et al., 1991)",Despite the establishment of Toll as a regulat...,None,A brief history of Toll
16,"(Thisse et al., 1991;",Dorsal was then able to regulate maternal-effe...,None,A brief history of Toll
17,"Pan et al., 1991)",Dorsal was then able to regulate maternal-effe...,None,A brief history of Toll
18,"(Nonaka et al., 1990;","At the time, it was known that in mammalian sy...",None,A brief history of Toll


##### Consolidate AnyStyle with Grobid

In [81]:
#Get AnyStyle Dictionary

with open('ref.json', 'r') as json_file:
    AnyStyle_data = json.load(json_file)

# Get Grobid Dictionary
grobid_data = [p.to_dict() for p in grob_doc.citations]
